In [1]:
from IPython import parallel
clients = parallel.Client('W:/ipcontroller-client.json')
clients.block = True  # use synchronous computations

c:\users\student\anaconda2\envs\pp3\lib\site-packages\IPython\parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated since IPython 4.0. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)


AttributeError: Client

In [ ]:
%px import os, matplotlib, json, gc
%px matplotlib.use('Agg')
%px import numpy as np
%px import pandas as pd
%px import matplotlib.pyplot as plt
%px from skimage import measure
%px from scipy import ndimage , misc, stats
%px matplotlib.style.use('ggplot')



In [ ]:
import os, matplotlib, json, gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage import measure
from scipy import ndimage , misc, stats
matplotlib.style.use('ggplot')



In [ ]:
def Collect_Pandas(directorylist):
    container = pd.DataFrame()
    container = container.append([pd.read_pickle(file) for file in directorylist])
    return container

In [ ]:
def generatefigures(directory,name):
    total_panda = Collect_Directory_Pandas(directory)
    total_panda.to_excel(name)
    total_panda
    mpld3.fig_to_html

In [ ]:
def filterObjectsdebug(labels,lower=20,upper=500):
    objects = ndimage.measurements.find_objects(labels)
    selectors=[]
    remove_objects =[]
    keep_objects=[]
    [selectors.append(labels[obj].size>lower and labels[obj].size<upper) for obj in objects]
    selectors=np.array(selectors)
    indexer = np.arange(selectors.size)
    indexer_inverse = np.arange(selectors.size)
    indexer = indexer[selectors];
    indexer_inverse = indexer_inverse[np.invert(selectors)];
    [remove_objects.append(objects[o]) for o in indexer_inverse];
    [keep_objects.append(objects[o]) for o in indexer];
    for remv in remove_objects:
        labels[remv] =0
    
    return labels,keep_objects,remove_objects 

In [ ]:

def filterObjects(labels,lower=20,upper=500):
    objects = ndimage.measurements.find_objects(labels)
    selectors=[]
    remove_objects =[]
    #keep_objects=[]
    [selectors.append(labels[obj].size>lower and labels[obj].size<upper) for obj in objects]
    selectors=np.array(selectors)
    indexer = np.arange(selectors.size)
    indexer_inverse = np.arange(selectors.size)
    indexer = indexer[selectors];
    indexer_inverse = indexer_inverse[np.invert(selectors)];
    [remove_objects.append(objects[o]) for o in indexer_inverse];
    #[keep_objects.append(objects[o]) for o in indexer];
    for remv in remove_objects:
        labels[remv] =0
    objects = ndimage.measurements.find_objects(labels)
    keep_objects = [x for x in objects if x] 
    
    return labels,keep_objects 

In [ ]:
class detectionobject:
    
    def detectlabels (self,array):
        labeled = measure.label(array)
        return labeled
    
    def detectobjects (self,labels):
        objects = ndimage.measurements.find_objects(labels)
        return objects

    def arealist (self,array,objects):
        areas = []
        [areas.append(array[obj].size) for obj in objects]
        return areas
    def __init__(self, array):
        self.labeled = self.detectlabels (array)
        self.objects = self.detectobjects(self.labeled)
        self.objectareas = self.arealist(array, self.objects)
        self.largestobjectslice = self.objects[np.nonzero(np.array(self.objectareas)== np.array(self.objectareas).max())[0][0]]
        self.largestobjectarray = array[self.largestobjectslice]

In [ ]:

class Paul_Allen_Atlas_object:
    
    def loadallen(self, rawfile,dtype = np.int32,size = (528,320,456)):
        with open(rawfile, 'rb') as fid:
            data_array = np.fromfile(fid, dtype)
        data_array = data_array.reshape(size, order="F")
        return data_array
   
        
    def __init__(self,path):
        self.ANO = self.loadallen(path)
        self.ANO_Threshold = self.ANO>0
        self.ANO_CenterMerge = np.copy(self.ANO_Threshold)
        
        Merge = self.ANO_Threshold [:,:,80:230].mean(2).transpose() * np.ones_like (self.ANO_Threshold[:,:,80:230]).transpose()
        self.ANO_CenterMerge[:,:,80:230] = Merge.transpose()


In [ ]:
#Code to identify ROI names

def build_structure(data, d=[]):
    if 'children' in data:
        for c in data['children']:
            d.append({'atlas_id': c.get('atlas_id' ),'name': c.get('name', None), 'id': c.get('id'),'ontology_id': c.get('ontology_id'),'parent_structure_id': c.get('parent_structure_id')})
            build_structure(c, d)
    return d

def identifystructure(labelnum,level = 0):  
    jdata = json.loads(open ('C:/Users/dfpena/Documents/Python/structures.json').read())
    u = build_structure(jdata['msg'][0])
    for dic in u:
        if dic['id']==labelnum:
            struc = dic 
    return struc


In [ ]:
def detect_all_Allen_threshold(path):
    allen = Paul_Allen_Atlas_object('C:/Users/dfpena/Documents/P56_Mouse_annotation/annotation.raw')
    thresh = allen.ANO_Threshold
    merged = allen.ANO_CenterMerge
    del allen
    c = []
    [c.append(thresh[x,:,:][detectionobject(merged[x,:,:]).largestobjectslice]) for x in np.arange(merged.shape[0])];
    del thresh
    del merged
    return pd.DataFrame(c)

In [ ]:
def detect_all_Allen_slices(path = 'C:/Users/dfpena/Documents/P56_Mouse_annotation/annotation.raw'):
    allen = Paul_Allen_Atlas_object(path)
    thresh = allen.ANO_Threshold
    merged = allen.ANO_CenterMerge
    del allen
    c = []
    [c.append(detectionobject(merged[x,:,:]).largestobjectslice) for x in np.arange(merged.shape[0])];
    del thresh
    del merged
    return pd.DataFrame(c)

In [ ]:
def detect_all_Allen_annotation(path = 'C:/Users/dfpena/Documents/P56_Mouse_annotation/annotation.raw'):
    allen = Paul_Allen_Atlas_object(path)
    ANO = allen.ANO
    thresh = allen.ANO_Threshold
    merged = allen.ANO_CenterMerge
    del allen
    c = []
    [c.append(ANO[x,:,:][detectionobject(merged[x,:,:]).largestobjectslice]) for x in np.arange(merged.shape[0])];
    del thresh
    del merged
    return pd.DataFrame(c)

In [ ]:
def _zoom2Large (largearray,smallarray):
        largearrayshape = np.float64(largearray.shape)
        smallarrayshape = np.float64 (smallarray.shape)
        zoomfactor = (largearrayshape [0]/smallarrayshape[0],largearrayshape[1]/smallarrayshape[1])
        zoomfactor = zoomfactor
        zoomedsmall = ndimage.zoom(smallarray, order=0, zoom = zoomfactor)
        return zoomedsmall

In [ ]:
def allencomparisonarray (sectionnumber, imagearray, Allen_detect_annotation_path='/home/dfpena/Documents/P56_Mouse_annotation/' ):
    Allendetected = pd.read_pickle(Allen_detect_annotation_path +'Allen_detected_annotation.panda').values[sectionnumber][0]
    largeallen = _zoom2Large(imagearray,Allendetected)
    return largeallen

    

In [ ]:
def AutoCrop(image, skip=1):
    Sectionbinary = image.sum(2)>image.sum(2).mean()
    detectedSection = detectionobject(Sectionbinary)
    if skip == 0:
        if (float(np.array(detectedSection.objectareas).max())/float (np.size(Sectionbinary)))>0.20:
            return image[detectedSection.largestobjectslice]
        else:
            return image
    else:
        return image
    

In [ ]:
def generatecelllocationarray(structureslist,sectionnumber,Regionnames,parentid,parentname):
    cellmap = pd.DataFrame({'SectionNumber': np.array([sectionnumber] * len(structureslist)),
                            'StructureID': structureslist[:,1],
                            'Side': structureslist[:,2],
                            'StructureName': Regionnames,
                            'parent_structure_id': parentid,
                            
                            
                           })
    return cellmap

In [ ]:
def filterImage(imagearray):
    imagearray = ndimage.filters.gaussian_filter(imagearray,2)
    thresh = imagearray>(imagearray.mean() + 4.5*imagearray.std())
    #imagearray[np.invert(thresh)] = 0
    #imagearray[thresh] = imagearray[thresh]>(imagearray[thresh].mean() + 1.5*imagearray[thresh].std()) 
    #return  imagearray
    if thresh.sum()<30:
        thresh = imagearray>(imagearray.mean() + 3.5*imagearray.std()) 
    return thresh

In [ ]:
def filterAllenMapbySingle(allenmap,conversionkeyword,startkeyword,identifier):
    filtered = allenmap[conversionkeyword].va\0lues[np.nonzero(allenmap[[startkeyword]].values==identifier)[0]]
    return filtered

In [ ]:
def filterAllenMapby(allenmap,conversionkeyword,startkeyword,identifierIter):
    filtered = [allenmap[conversionkeyword].values[np.nonzero(allenmap[[startkeyword]].values==identifier)[0][0]] for identifier in identifierIter]
    return filtered



    

In [ ]:
def enumerateRegions(objects,ResizedAllen,Sectionnumber,allenpandamappath ='/home/dfpena/Documents/P56_Mouse_annotation/P56_Mouse_annotation/'):
    allenmap = pd.read_pickle(allenpandamappath + 'Allen_Lookup.panda')
    AllenRegion=[]
    [AllenRegion.append([obj[1].start,stats.mode(ResizedAllen[obj].flatten())[0][0].astype('int'),-1]) for obj in objects];
    AllenRegion= np.array(AllenRegion)
    AllenRegion= AllenRegion[AllenRegion[:,1] > 0]
    AllenRegion= AllenRegion[AllenRegion[:,1] != 8]
    selector_range = np.arange(AllenRegion.shape[0])
    bool_selector =  (AllenRegion[:,0] < ResizedAllen.shape[1]/2) 
    AllenRegion[selector_range[bool_selector ],2]= 0
    AllenRegion[selector_range[np.invert(bool_selector) ],2] = 1
    Regionnames = filterAllenMapby(allenmap,'name','id',AllenRegion[:,1])
    parentid = filterAllenMapby(allenmap,'parent_structure_id','id',AllenRegion[:,1])
    parentname = filterAllenMapby(allenmap,'name','id',parentid)
    grandparentid = filterAllenMapby(allenmap,'parent_structure_id','id',parentid)
    grandparentname = filterAllenMapby(allenmap,'name','id',grandparentid)
    cellmap = generatecelllocationarray(AllenRegion,Sectionnumber,Regionnames,parentid,parentname)
    return cellmap

In [ ]:
def filterdirectory(path,extension):
    directory= [file for file in os.listdir(path) if file.lower().endswith(extension) and file[0] !='.']
    return directory

In [ ]:
def mkdirsafe (newpath):
    if not os.path.exists(newpath): os.makedirs(newpath)

In [ ]:
def mkdebug_fig(folder,name,fig, alphav=1):
    mkdirsafe(folder)
    plt.imshow(fig,alpha=alphav)
    plt.savefig(folder+'/'+name)


In [ ]:
def process_images(ipath,iterator,allenlibpath):
    try:    
        mkdirsafe('panda')
        mkdirsafe('3d')
        mkdirsafe('arrays')
        Sectionnumber = int(ipath.split('_')[0])
        image = AutoCrop(ndimage.imread(ipath))
        np.savez('arrays/'+str(iterator)+'_'+str(Sectionnumber)+'_'+ 'croppedsection',image)
        image = filterImage(image[:,:,1])
        np.savez('arrays/'+str(iterator)+'_'+str(Sectionnumber)+'_'+'filtercroppedsection',image)
        labeled = detectionobject(image).labeled
        labeled, objects = filterObjects(labeled)
        np.savez('arrays/'+str(iterator)+'_'+str(Sectionnumber)+'_'+ 'labels',labeled)
        #mkdebug_fig('DebugImages',(str(Sectionnumber)+'_labeledfiltered'),labeled)
        Allen = allencomparisonarray(Sectionnumber,image,allenlibpath)
        np.savez('arrays/'+str(iterator)+'_'+str(Sectionnumber)+'_'+'AllenResized',Allen)
        #mkdebug_fig('DebugImages',str(iterator) + '_' +(str(Sectionnumber)+'_Verification'),Allen,0.6)
        cellmap =enumerateRegions(objects,Allen,Sectionnumber,allenlibpath)
        cellmap.to_pickle('panda/'+ str(iterator) + '_' +str(Sectionnumber)+'.panda')
        np.savez('3d/'+str(iterator)+'_'+str(Sectionnumber),objects )
        print(iterator, ipath)
        return (iterator, ipath)
    except ValueError:
        print('No cells could be detected')
        print(iterator, ipath)
        return (iterator, ipath)
    
        
    
    

In [ ]:
def load_functions_allcores(path, dcview):
    dcview['mkdirsafe'] =mkdirsafe
    dcview['mkdebug_fig'] =mkdebug_fig
    dcview['AutoCrop'] = AutoCrop
    dcview['detectionobject'] = detectionobject
    dcview['filterImage'] = filterImage
    dcview['filterObjects'] = filterObjects
    dcview['allencomparisonarray'] = allencomparisonarray
    dcview['_zoom2Large'] = _zoom2Large
    dcview['enumerateRegions'] = enumerateRegions
    dcview['filterAllenMapby'] = filterAllenMapby
    dcview['generatecelllocationarray'] = generatecelllocationarray
    dcview["os.chdir('{}') ".format(path)]
    
    

In [ ]:
def Process_images_SuperCluster(view, imagelist,imagenumberarray,path_to_allenlib='W:/Allen/'):
    """create a list of arguments for process images and distribute to cores"""
    path_to_allenlib_iterator = []
    [path_to_allenlib_iterator.append(path_to_allenlib) for i in np.arange(np.size(imagelist))]
    return view.map(process_images,imagelist,imagenumberarray,path_to_allenlib_iterator)
    
    

In [ ]:
def figure_from_pickle(picklepath):
    figure = np.load(picklepath)['arr_0']
    name = picklepath.split('.')
    mkdebug_fig('DebugImages',(name[0]),figure)
    plt.close()
    gc.collect()
    
    

In [ ]:
def mkexcel( path_to_pandas, animalname):
    os.chdir(path_to_pandas)
    directory = filterdirectory(os.curdir,".panda")

    Brain = Collect_Pandas(directory)
    LateralizedDF = Brain.groupby(['Side'])
    LeftDF = LateralizedDF.get_group(0)
    RightDF = LateralizedDF.get_group(1)


    TotalCountsDF = pd.DataFrame({'Cells':Brain.StructureName.value_counts()})
    LeftCountsDF = pd.DataFrame({'Cells':LeftDF.StructureName.value_counts()})
    RightCountsDF = pd.DataFrame({'Cells':RightDF.StructureName.value_counts()})


    os.chdir('..')
    mkdirsafe('ExcelSheets')
    #Raw
    Brain.to_pickle('ExcelSheets/'+animalname +'_total.compile')
    Brain.to_excel('ExcelSheets/'+animalname +'_RawTotal.xls')
    LeftDF.to_excel ('ExcelSheets/'+animalname +'_RawLeft.xls')
    RightDF.to_excel ('ExcelSheets/'+animalname +'_RawRight.xls')



    #Compiled
    TotalCountsDF.to_excel ('ExcelSheets/'+animalname +'_TotalFreq.xls')
    LeftCountsDF.to_excel ('ExcelSheets/'+animalname +'_LeftFreq.xls')
    RightCountsDF.to_excel ('ExcelSheets/'+animalname +'_RightFreq.xls')
    



In [ ]:
def mkmpld3figs (path_to_ExcelSheetsfolder,animalname):
    import mpld3
    from mpld3 import plugins
    os.chdir(path_to_ExcelSheetsfolder)
    TotalCountsDF = pd.read_excel(animalname+'_TotalFreq.xls')
    fig = plt.figure(figsize=(10,5))
    bars = plt.bar(np.arange(TotalCountsDF.size),TotalCountsDF.values)
    for i, bar in enumerate(bars.get_children()):
        tooltip = mpld3.plugins.LineLabelTooltip(bar, label=str(TotalCountsDF.index[i]))
        mpld3.plugins.connect(plt.gcf(), tooltip)
    plt.xlim(0,50)
    plt.title('Cell Detection for : Subject {0}'.format(animalname))
    plt.ylabel('Cell Count')
    plt.xlabel('Region number (Scroll over Bar for Region Name)')

    a = mpld3.fig_to_html(fig)


    Html_file= open("{0}.html".format(animalname),"w")
    Html_file.write(a)
    Html_file.close()
    plt.savefig('{0}.svg'.format(animalname))


In [ ]:
def mke_all_directories(path):
    os.chdir(path)
    mkdirsafe('panda')
    mkdirsafe('3d')
    mkdirsafe('arrays')
    mkdirsafe('ExcelSheets')

In [ ]:
def EasyRun_process_folder_images_Supercluster_balanced(animalname,path_to_images,allenlib,clients):
    if path_to_images[-1] =="/":
        path_to_figurearray = path_to_images + "arrays/"
    else:
        path_to_figurearray = path_to_images + "/arrays/"
        
    mke_all_directories(path_to_images)
    os.chdir(path_to_images)
    
    directory = filterdirectory(os.curdir,".jpg")
    load_functions_allcores(folder,clients[:])
    balanced = clients.load_balanced_view()
    Process_images_SuperCluster(balanced,directory,np.arange(np.size(directory)))
    mkexcel(path_to_images + 'panada', animalname)
    mkmpld3figs (path_to_images + 'ExcelSheets', animalname)
    
    os.chdir(path_to_figurearray)
    load_functions_allcores(path_to_figurearray,clients[:])
    pickledirectory = filterdirectory(os.curdir,".npz")
    output= balanced.map(figure_from_pickle, pickledirectory)
    

In [ ]:
def mkedebugfigsParallel(path_to_figurearray, clients):
    balanced = clients.load_balanced_view()
    os.chdir(path_to_figurearray)
    load_functions_allcores(path_to_figurearray,clients[:])
    pickledirectory = filterdirectory(os.curdir,".npz")
    output = balanced.map(figure_from_pickle,pickledirectory)
    return output

    
    

In [ ]:
def Determine_missing_panda(imagepath,pandapath,filters =['.jpg','.panda']):
    a = filterdirectory(imagepath,filters[0])
    b = filterdirectory(pandapath,filters[1])
    imagenumbersarray = [i.split("_")[0] for i in a]
    pandacompleted = [i.split("_")[1].split('.')[0] for i in b]
    return list(set(imagenumbersarray) -set(pandacompleted))


In [ ]:
def regenerateruntimelist (missingpandaslist,firstrundirectorylist):
    directoryframe = pd.DataFrame([i.split('_') for i in firstrundirectorylist])
    indexlist = [directoryframe.loc[directoryframe[0]==i].index.get_values()[0] for i in missingpandaslist]
    return indexlist

In [ ]:
def RetryErrorImages (imagepath,allenlib):
    os.chdir(imagepath)
    pandapath = imagepath +'panda/'
    a = Determine_missing_panda(imagepath,pandapath)
    b = filterdirectory(imagepath,'.jpg')
    c = regenerateruntimelist(a,b)
    out = [process_images(b[i],i,path_to_allenlib) for i in c]
    return out

    

TYPICAL RUN 1

In [ ]:
path_to_images = 'W:/Confirmed Images/D1D2 Chronic Pain/D2/18_fixed/'
path_to_allenlib = 'W:/Allen/'
path_to_figurearray= 'W:/Confirmed Images/D1D2 Chronic Pain/D2/18_fixed/arrays'
animalname = '18'

os.chdir(path_to_images) 
directory = filterdirectory(os.curdir,".jpg")

load_functions_allcores(path_to_images,clients[:])
balanced = clients.load_balanced_view()

Process_images_SuperCluster(balanced, directory,np.arange(np.size(directory)))

mkexcel(path_to_images +'panda', animalname)
mkmpld3figs(path_to_images +'ExcelSheets',animalname)

os.chdir(path_to_figurearray)
load_functions_allcores(path_to_figurearray,clients[:])
pickledirectory = filterdirectory(os.curdir,".npz")
output = balanced.map(figure_from_pickle,pickledirectory)

In [ ]:
path_to_images = 'W:/Confirmed Images/D1D2 Chronic Pain/D2/CPT5 11 Cuff/'
path_to_allenlib = 'W:/Allen/'
path_to_figurearray= 'W:/Confirmed Images/D1D2 Chronic Pain/D2/CPT5 11 Cuff/arrays'
animalname = '11'

os.chdir(path_to_images) 
directory = filterdirectory(os.curdir,".jpg")

load_functions_allcores(path_to_images,clients[:])
balanced = clients.load_balanced_view()

Process_images_SuperCluster(balanced, directory,np.arange(np.size(directory)))




mkexcel(path_to_images +'panda', animalname)
mkmpld3figs(path_to_images +'ExcelSheets',animalname)



os.chdir(path_to_figurearray)
load_functions_allcores(path_to_figurearray,clients[:])
pickledirectory = filterdirectory(os.curdir,".npz")
output = balanced.map(figure_from_pickle,pickledirectory)